In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [6]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
               'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
               'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
               'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset["classification_yes"]

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
indep = sc.fit_transform(indep)

In [8]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

param_grid= {'criterion':['gini', 'entropy'],
             'max_features':['sqrt','log2'],
             'splitter':['best','random']}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit=True, verbose=3, cv=5, n_jobs=-1, scoring='f1_weighted')

#fitting the model for Grid search
grid.fit(indep, dep)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [9]:
re=grid.cv_results_

table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012792,0.006043,0.008795,0.000400,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.827778,0.975000,0.949628,0.975148,0.866620,0.918835,0.060484,7
1,0.004996,0.002529,0.008595,0.001019,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.889274,0.962618,1.000000,0.885065,0.935368,0.934465,0.043751,6
2,0.005197,0.001326,0.012193,0.006397,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.709396,0.987456,0.913230,0.950521,0.880815,0.888284,0.096320,8
3,0.005596,0.003381,0.008995,0.002097,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.962813,0.975148,0.974814,0.975000,1.000000,0.977555,0.012173,1
4,0.005196,0.000748,0.010795,0.001720,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.913230,0.938259,0.975000,1.000000,0.974500,0.960198,0.030644,3
5,0.003925,0.000144,0.008595,0.000799,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.889143,0.987539,0.987456,0.987539,0.935983,0.957532,0.039592,4
6,0.005996,0.002607,0.010994,0.002606,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.913230,0.962813,0.937280,0.938022,0.948552,0.939979,0.016257,5
7,0.005744,0.002441,0.008795,0.001469,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.938259,0.950296,1.000000,0.975000,0.961895,0.965090,0.021288,2


In [10]:
grid_predictions = grid.predict(indep)

In [11]:
print("Best parameters found: ", grid.best_params_)

Best parameters found:  {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'random'}


In [12]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(dep, grid_predictions)    

In [13]:
print(cm)

[[150   0]
 [  0 249]]


In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(dep, grid_predictions)

In [15]:
print(clf_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399



In [16]:
from sklearn.metrics import roc_auc_score

roc_auc_score(dep,grid.predict_proba(indep)[:,1])

1.0